In [1]:
import pandas as pd
import numpy as np
import talib as ta

In [9]:
df_train = pd.read_feather('./data'+'/new_data.ftr', 
                           columns=["timestamp",'Asset_ID', 'Count', 'Open', 'High', 'Low', 'Close', 'Volume', 'Target',
                                    'Weight', 'lr_15', 'Mkt_lrt_15', 'Crypto_Index','beta','lr_mkt_resid'])
df_train.rename(columns={"beta": "beta_60m","lr_mkt_resid":"lr_mkt_resid_60m"},inplace=True)
df_train.set_index(["timestamp",'Asset_ID'], inplace=True)
df_train

Count          Open          High           Low  \
timestamp  Asset_ID                                                    
1514764860 2          40.0   2376.580000   2399.500000   2357.140000   
           0           5.0      8.530000      8.530000      8.530000   
           1         229.0  13835.194000  14013.800000  13666.110000   
           5          32.0      7.659600      7.659600      7.656700   
           7           5.0     25.920000     25.920000     25.874000   
...                    ...           ...           ...           ...   
1641772800 9         218.0    130.842571    130.940000    130.478000   
           10         20.0   2122.416667   2124.010000   2116.950000   
           13        118.0      0.066109      0.066130      0.066058   
           12        112.0      0.261055      0.261185      0.260474   
           11         52.0    190.732000    191.160000    189.800000   

                            Close        Volume    Target    Weight     lr_15  \
timestamp  Asset_ID                                                             
1514764860 2          2374.590000  1.923301e+01 -0.004218  0.058657       NaN   
           0             8.530000  7.838000e+01 -0.014399  0.105286       NaN   
           1         13850.176000  3.155006e+01 -0.014643  0.165850       NaN   
           5             7.657600  6.626713e+03 -0.013922  0.033911       NaN   
           7            25.877000  1.210873e+02 -0.008264  0.050867       NaN   
...                           ...           ...       ...       ...       ...   
1641772800 9           130.707429  1.011042e+03       NaN  0.058657 -0.004696   
           10         2119.743333  2.084755e+00       NaN  0.026874 -0.003462   
           13            0.066082  1.265238e+06       NaN  0.043830 -0.003493   
           12            0.260682  1.187581e+05       NaN  0.050867 -0.003568   
           11          190.506000  1.355932e+02       NaN  0.039370 -0.000454   

                     Mkt_lrt_15  Crypto_Index  beta_60m  lr_mkt_resid_60m  
timestamp  Asset_ID                                                        
1514764860 2           0.000000   2571.477256  0.000000               NaN  
           0           0.000000   2571.477256  0.000000               NaN  
           1           0.000000   2571.477256  0.000000               NaN  
           5           0.000000   2571.477256  0.000000               NaN  
           7           0.000000   2571.477256  0.000000               NaN  
...                         ...           ...       ...               ...  
1641772800 9          -0.003263   7532.261935  0.963412         -0.001552  
           10         -0.003263   7532.261935  1.507092          0.001455  
           13         -0.003263   7532.261935  0.772527         -0.000973  
           12         -0.003263   7532.261935  0.938556         -0.000505  
           11         -0.003263   7532.261935  0.576140          0.001426  

[26473302 rows x 13 columns]

In [10]:
df_train.columns

Index(['Count', 'Open', 'High', 'Low', 'Close', 'Volume', 'Target', 'Weight',
       'lr_15', 'Mkt_lrt_15', 'Crypto_Index', 'beta_60m', 'lr_mkt_resid_60m'],
      dtype='object')

Weighted info `lr_mkt_resid` similar to `Target` which can remove the market signal from individual asset returns. 

$$R^{a}(t)=\log P^{a}_t - \log P^{a}_{t-15}$$

$$M(t)=\frac{\sum_{a} w^{a} R^{a}(t)}{\sum_{a} w^{a}}$$

$$\beta^{a}=\frac{\left\langle M \cdot R^{a}\right\rangle_{60}}{\left\langle M^{2}\right\rangle_{60}}$$

$$lrmktresid^{a}(t)=R^{a}(t)-\beta^{a} M(t)$$

where the bracket  $⟨.⟩_{60}$ represent the rolling average over time (60 minute windows).

In [13]:
def beta(df, window=60*24*2): 
    b = (ta.MULT(df.Mkt_lrt_15,df.lr_15).rolling(window).mean())/(ta.MULT(df.Mkt_lrt_15,df.Mkt_lrt_15).rolling(window).mean())
    return b

def lr_mkt_resid(df):
    return ta.SUB(df.lr_15, ta.MULT(df.beta_2d, df.Mkt_lrt_15))

# def rolling_volatility(df, window=60):
#     return df.rolling(window).std()

# def lrtn_index(crypto_index):
#     lrtn_index_5 = np.log(crypto_index).diff(5).rename('lrtn_index_5')
#     lrtn_index_15 = np.log(crypto_index).diff(15).rename('lrtn_index_15')
#     return pd.concat([lrtn_index_5,lrtn_index_15],axis=1)

# def lr_mkt_resid_sma(lr_mkt_resid):
#     return ta.SMA(lr_mkt_resid,5)
#############################################
def lambda_by_Asset(x):
    ##beta
    b = beta(x).rename("beta_2d")
    b = b.replace([np.nan,np.inf,-np.inf], 0)
    x= x.merge(b, on =['timestamp','Asset_ID'],how='left')
    ##
    x['lr_mkt_resid_2d'] = lr_mkt_resid(x)
    return x

In [23]:
beta(df_train.loc[df_train.index.get_level_values('Asset_ID') == 0]).rename("beta_2d")

timestamp   Asset_ID
1514764860  0                NaN
1514764920  0                NaN
1514764980  0                NaN
1514765040  0                NaN
1514765100  0                NaN
                          ...   
1641772560  0           0.892210
1641772620  0           0.892218
1641772680  0           0.892261
1641772740  0           0.892208
1641772800  0           0.892242
Name: beta_2d, Length: 2102319, dtype: float64

In [14]:
#######################################add features by asset_id
df_train = df_train.groupby('Asset_ID').apply(lambda x: lambda_by_Asset(x))
df_train.iloc[-14:,]


Count          Open          High           Low  \
timestamp  Asset_ID                                                     
1641772800 3          351.0      1.170103      1.170729      1.167000   
           2          106.0    376.696667    376.950000    376.000000   
           0          288.0    438.722333    438.800000    437.700000   
           1         2111.0  41860.168571  41877.700000  41766.700000   
           4          197.0      0.150498      0.150600      0.150200   
           5           88.0      2.794480      2.800000      2.788562   
           7           75.0     29.731780     29.751000     29.698200   
           6         2059.0   3151.504286   3152.150000   3143.790000   
           8           29.0      1.034633      1.075000      0.957800   
           9          218.0    130.842571    130.940000    130.478000   
           10          20.0   2122.416667   2124.010000   2116.950000   
           13         118.0      0.066109      0.066130      0.066058   
           12         112.0      0.261055      0.261185      0.260474   
           11          52.0    190.732000    191.160000    189.800000   

                            Close        Volume  Target    Weight     lr_15  \
timestamp  Asset_ID                                                           
1641772800 3             1.167863  1.692235e+05     NaN  0.107797 -0.005190   
           2           376.480000  1.443180e+02     NaN  0.058657 -0.002764   
           0           437.990333  4.016819e+02     NaN  0.105286 -0.002281   
           1         41823.085714  8.982209e+01     NaN  0.165850 -0.002324   
           4             0.150307  1.145598e+06     NaN  0.086971 -0.003500   
           5             2.789640  2.411353e+04     NaN  0.033911 -0.005577   
           7            29.715100  1.063943e+03     NaN  0.050867 -0.003969   
           6          3147.158571  5.786272e+02     NaN  0.144188 -0.003110   
           8             1.034400  8.585877e+03     NaN  0.026874 -0.001899   
           9           130.707429  1.011042e+03     NaN  0.058657 -0.004696   
           10         2119.743333  2.084755e+00     NaN  0.026874 -0.003462   
           13            0.066082  1.265238e+06     NaN  0.043830 -0.003493   
           12            0.260682  1.187581e+05     NaN  0.050867 -0.003568   
           11          190.506000  1.355932e+02     NaN  0.039370 -0.000454   

                     Mkt_lrt_15  Crypto_Index  beta_60m  lr_mkt_resid_60m  \
timestamp  Asset_ID                                                         
1641772800 3          -0.003263   7532.261935  1.345836         -0.000798   
           2          -0.003263   7532.261935  0.941773          0.000309   
           0          -0.003263   7532.261935  0.753021          0.000177   
           1          -0.003263   7532.261935  1.083834          0.001212   
           4          -0.003263   7532.261935  0.789767         -0.000923   
           5          -0.003263   7532.261935  0.891223         -0.002668   
           7          -0.003263   7532.261935  0.728955         -0.001591   
           6          -0.003263   7532.261935  0.972044          0.000062   
           8          -0.003263   7532.261935  2.351759          0.005774   
           9          -0.003263   7532.261935  0.963412         -0.001552   
           10         -0.003263   7532.261935  1.507092          0.001455   
           13         -0.003263   7532.261935  0.772527         -0.000973   
           12         -0.003263   7532.261935  0.938556         -0.000505   
           11         -0.003263   7532.261935  0.576140          0.001426   

                      beta_2d  lr_mkt_resid_2d  
timestamp  Asset_ID                             
1641772800 3         1.282272        -0.001006  
           2         0.898460         0.000168  
           0         0.892242         0.000631  
           1         0.717869         0.000018  
           4         0.953432        -0.000389  
           5

In [15]:
df_train.reset_index().to_feather('./data/new_data2.ftr')